In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)
        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((325, 224, 224, 3), (82, 224, 224, 3), (325,), (82,))

In [5]:
X_train = X_train / 255
X_test = X_test / 255

In [6]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([ 75,  77, 173], dtype=int64))

In [7]:
# y_train.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts

(array([0, 1, 2]), array([27, 20, 35], dtype=int64))

In [8]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train.shape, y_test.shape

((325, 3), (82, 3))

In [10]:
###############

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the image width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the image height
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True    # Random horizontal flipping
)

In [9]:
from tensorflow.keras import layers, models

# Define the CNN architecture
def create_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [10]:
# Create the CNN model
input_shape = X_train.shape[1:]
num_classes = 3
model = create_cnn(input_shape, num_classes)

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, 
                    batch_size=64, validation_data=(X_test, y_test))

# # Fit the model with data augmentation
# batch_size = 64
# epochs = 10

# history = model.fit(train_datagen.flow(X_train, y_train, batch_size=batch_size),
#                     epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
5/5 [==============================] - 19s 4s/step - loss: 0.8412 - accuracy: 0.6361 - val_loss: 0.8542 - val_accuracy: 0.5784
Epoch 2/10
5/5 [==============================] - 20s 4s/step - loss: 0.7256 - accuracy: 0.6721 - val_loss: 0.7327 - val_accuracy: 0.6078
Epoch 3/10
5/5 [==============================] - 21s 4s/step - loss: 0.6841 - accuracy: 0.6820 - val_loss: 0.6612 - val_accuracy: 0.5980
Epoch 4/10
5/5 [==============================] - 19s 4s/step - loss: 0.5427 - accuracy: 0.7672 - val_loss: 0.6451 - val_accuracy: 0.6373
Epoch 5/10
5/5 [==============================] - 19s 4s/step - loss: 0.5125 - accuracy: 0.7607 - val_loss: 0.5956 - val_accuracy: 0.7255
Epoch 6/10
5/5 [==============================] - 17s 3s/step - loss: 0.5258 - accuracy: 0.7770 - val_loss: 0.5837 - val_accuracy: 0.7255
Epoch 7/10
5/5 [==============================] - 17s 3s/step - loss: 0.5108 - accuracy: 0.7377 - val_loss: 0.7256 - val_accuracy: 0.6275
Epoch 8/10
5/5 [==================

In [16]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

4/4 [==============================] - 2s 492ms/step - loss: 0.6753 - accuracy: 0.6667
Test loss: 0.6752800941467285
Test accuracy: 0.6666666865348816


In [9]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet')

In [10]:
# Freeze the convolutional base
base_model.trainable = False

In [11]:
# Add custom dense layers on top of the convolutional base
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Output layer with reduced size (e.g., 10 classes)
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1000)              8062504   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 256)               256256    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               3

In [13]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=25, 
                    batch_size=64, validation_data=(X_test, y_test))

Epoch 1/25
6/6 [==============================] - 29s 4s/step - loss: 1.0930 - accuracy: 0.4523 - val_loss: 1.0866 - val_accuracy: 0.4268
Epoch 2/25
6/6 [==============================] - 24s 4s/step - loss: 1.0687 - accuracy: 0.5323 - val_loss: 1.0758 - val_accuracy: 0.4268
Epoch 3/25
6/6 [==============================] - 23s 4s/step - loss: 1.0405 - accuracy: 0.5323 - val_loss: 1.0623 - val_accuracy: 0.4268
Epoch 4/25
6/6 [==============================] - 24s 4s/step - loss: 1.0017 - accuracy: 0.5323 - val_loss: 1.0566 - val_accuracy: 0.4268
Epoch 5/25
6/6 [==============================] - 24s 4s/step - loss: 0.9638 - accuracy: 0.5323 - val_loss: 1.0579 - val_accuracy: 0.4268
Epoch 6/25
6/6 [==============================] - 24s 4s/step - loss: 0.9436 - accuracy: 0.5323 - val_loss: 1.0476 - val_accuracy: 0.4268
Epoch 7/25
6/6 [==============================] - 24s 4s/step - loss: 0.9136 - accuracy: 0.5323 - val_loss: 0.9970 - val_accuracy: 0.4268
Epoch 8/25
6/6 [==================

In [15]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

3/3 [==============================] - 4s 1s/step - loss: 0.5443 - accuracy: 0.8049
Test loss: 0.5442630648612976
Test accuracy: 0.8048780560493469
